In [ ]:
import pandas as pd

path = "/kaggle/input/competitions/titanic/train.csv"
df = pd.read_csv(path)

In [ ]:
df.columns.tolist()

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [ ]:
df.isna().any()

PassengerId    False
Survived       False
Pclass         False
Name           False
Sex            False
Age             True
SibSp          False
Parch          False
Ticket         False
Fare           False
Cabin           True
Embarked        True
dtype: bool

In [ ]:
def fillna_with_mean(df, cols):
    df[cols] = df[cols].fillna(value=df[cols].mean())
    return df

In [2]:
from utils import fillna_with_mean

ModuleNotFoundError: No module named 'utils'

In [39]:
cd ..

/


In [40]:
ls

bin@                        lib@                      requirements.txt
boot/                       lib32@                    root/
colab_requirements.txt      lib64@                    run/
content/                    libx32@                   sbin@
cuda-keyring_1.1-1_all.deb  media/                    srv/
datalab/                    mnt/                      sys/
dev/                        NGC-DL-CONTAINER-LICENSE  tmp/
etc/                        opt/                      tools/
home/                       proc/                     usr/
kaggle/                     python-apt/               var/
kaggle_requirements.txt     python-apt.tar.xz*


In [44]:
cd kaggle/ 


/kaggle


In [45]:
ls

input/  lib/  working/


In [1]:
cd /../kaggle/working

/kaggle/working


In [1]:
import sys
sys.path.append("/kaggle/working/titanic")

In [2]:
from utils import fillna_with_mean

ImportError: cannot import name 'fillna_with_mean' from 'utils' (/kaggle/working/titanic/utils.py)

In [7]:
import os
os.listdir()

['titanic', '.virtual_documents']